In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")


In [ ]:
chain = prompt | model

In [ ]:
test_text = """Jay Chou (simplified Chinese: 周杰伦; traditional Chinese: 周杰倫; pinyin: Zhōu Jiélún; born January 18, 1979)[1] is a Taiwanese singer, songwriter, record producer, rapper, actor, television personality, and businessman. Dubbed the "King of Mandopop", and having sold over 30 million records, Chou is one of the best-selling artists in Taiwan and is known for his work with lyricist Vincent Fang, with whom he has frequently collaborated on his music.[2]

In 2000, Chou released his debut studio album, Jay (2000), under the record company Alfa Music. Chou rose to fame with the release of his second studio album, Fantasy (范特西) (2001), which combined Western and Eastern music styles. The album won five Golden Melody Awards, including Album of the Year. He has since further released twelve more studio albums, spawning a string of hit singles and gaining significant prominence in Asian communities such as Taiwan, Hong Kong SAR, China Mainland, Malaysia, and Singapore. Chou has embarked on six world tours, performing in cities around the world to more than 10 million spectators as of 2019.[3]

In 2007, Chou established his own record and management company JVR Music.[4] Outside of music, Chou has served as the President of his own fashion brand PHANTACi since 2006. As an actor, Chou made his acting debut in the film Initial D 頭文字D(2005), followed shortly by a starring role in the epic Curse of the Golden Flower (2006). He has since starred in a number of movies, becoming known to Western audiences when he made his Hollywood debut in 2011 with The Green Hornet, starring alongside Seth Rogen and Christoph Waltz, followed by Now You See Me 2 (2016).[5]"""

In [ ]:
# Create the retriever
vectorstore = Chroma.from_texts([test_text], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [ ]:
chain.invoke("where did harrison work?")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"question": "where did harrison work", "language": "italian"})



In [ ]:
from langchain.schema.runnable import RunnableMap
from langchain.schema import format_document

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
def _combine_documents(docs, document_prompt = DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
from typing import Tuple, List
def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer

In [ ]:
_inputs = RunnableMap(
    {
        "standalone_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: _format_chat_history(x['chat_history'])
        } | CONDENSE_QUESTION_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser(),
    }
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"]
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [ ]:
conversational_qa_chain.invoke({
    "question": "where did harrison work?",
    "chat_history": [],
})

In [ ]:
conversational_qa_chain.invoke({
    "question": "where did he work?",
    "chat_history": [("Who wrote this notebook?", "Harrison")],
})

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(return_messages=True, output_key="answer", input_key="question")

In [ ]:
# First we add a step to load memory
# This needs to be a RunnableMap because its the first input
loaded_memory = RunnableMap(
    {
        "question": itemgetter("question"),
        "memory": memory.load_memory_variables,
    }
)
# Next we add a step to expand memory into the variables
expanded_memory = {
    "question": itemgetter("question"),
    "chat_history": lambda x: x["memory"]["history"]
}

# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x['chat_history'])
    } | CONDENSE_QUESTION_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"]
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question")
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | expanded_memory | standalone_question | retrieved_documents | answer

In [ ]:
inputs = {"question": "where did harrison work?"}
result = final_chain.invoke(inputs)
result

In [ ]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [ ]:
memory.load_memory_variables({})

In [ ]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

chain1 = prompt1 | model | StrOutputParser()

chain2 = {"city": chain1, "language": itemgetter("language")} | prompt2 | model | StrOutputParser()

chain2.invoke({"person": "obama", "language": "spanish"})

In [ ]:
from langchain.schema.runnable import RunnableMap
prompt1 = ChatPromptTemplate.from_template("generate a random color")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}")
prompt3 = ChatPromptTemplate.from_template("what is countries flag that has the color: {color}")
prompt4 = ChatPromptTemplate.from_template("What is the color of {fruit} and {country}")
chain1 = prompt1 | model | StrOutputParser()
chain2 = RunnableMap(steps={"color": chain1}) | {
    "fruit": prompt2 | model | StrOutputParser(),
    "country": prompt3 | model | StrOutputParser(),
} | prompt4

In [ ]:
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field

class PromptToUse(BaseModel):
    """Used to determine which prompt to use to answer the user's input."""
    
    name: str = Field(description="Should be one of `math` or `english`")


In [ ]:
tagger = create_tagging_chain_pydantic(PromptToUse, ChatOpenAI(temperature=0))



In [ ]:
chain1 = ChatPromptTemplate.from_template("You are a math genius. Answer the question: {question}") | ChatOpenAI()
chain2 = ChatPromptTemplate.from_template("You are an english major. Answer the question: {question}") | ChatOpenAI()

In [ ]:
from langchain.schema.runnable import RouterRunnable
router = RouterRunnable({"math": chain1, "english": chain2})

In [ ]:
chain = {
    "key": {"input": lambda x: x["question"]} | tagger | (lambda x: x['text'].name),
    "input": {"question": lambda x: x["question"]}
} | router

In [ ]:
chain.invoke({"question": "whats 2 + 2"})

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

In [ ]:
search = DuckDuckGoSearchRun()